# A RAG system for the Kenyan Constitution

In [1]:
import pdfplumber
# import spacy
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import pipeline
import chromadb
from chromadb.utils import embedding_functions
import os
from dotenv import load_dotenv

load_dotenv()

ModuleNotFoundError: No module named 'faiss'